In [1]:
import matplotlib.pyplot as plt
%matplotlib inline 
import h5py
import numpy as np
import pandas as pd
from astropy.table import Table, Column, vstack
from astropy.io.misc.hdf5 import read_table_hdf5, write_table_hdf5

In [2]:
def getFiles(dirname, energy):
    import os
    import re
    (root, _, allfilenames) = next(os.walk(dirname))
    energy_str = '_{:.2f}_'.format(energy)
    regex = re.compile(re.escape(energy_str))
    
    filenames = [os.path.join(root, f) for f in allfilenames if regex.search(f)]
    
    return filenames  

def concatenate_h5(filenames):
    return [read_table_hdf5(file, 'table') for file in filenames]        

In [3]:
# get the energies
# We'll look at the Er values of the data points
# import data from Edelweiss
resNR_data = pd.read_csv("data/edelweiss_NRwidth_GGA3_data.txt", skiprows=1, \
                       names=['E_recoil', 'sig_NR', 'E_recoil_err', 'sig_NR_err'], \
                       delim_whitespace=True)

# the sorting is necessary!
# otherwise the mask defined below will select the wrong data
resNR_data = resNR_data.sort_values(by='E_recoil')
NR_data = {'Erecoil': resNR_data["E_recoil"][2::], 'sigma': resNR_data["sig_NR"][2::], 'sigma_err': resNR_data["sig_NR_err"][2::]}


In [5]:
# combine all the files for each energy
for Erecoil in NR_data['Erecoil']:
    # an array of strings
    filenames = getFiles('../../nrFano_data',Erecoil)
    table_arr = concatenate_h5(filenames)
    full_tab = vstack(table_arr)
    
    # don't overwrite if it already exists
    try:
        h5_filename = '../analysis_notebooks/data/yield_accuracy_Erecoil_%.2f_keV_%s.h5' % (Erecoil, 'all')
        full_tab.write(h5_filename, format='hdf5', path='table')
    except:
        pass

In [27]:
tab1 = read_table_hdf5(filenames[0], 'table')
tab2 = read_table_hdf5(filenames[0], 'table')

In [26]:
tab1

energy_recoil_keV,aH,C,m,scale,A,B,true_yield_sig,cor1_yield_sig
float64,float64,float64,float64,float64,float64,float64,float64,float64
24.5012,0.037259034286155075,0.03820656999918315,7.459537803714777e-05,0.9778066235294974,0.06887616665387132,0.1616482321094805,0.040563204164007674,0.04261415589080108
24.5012,0.03978345283249353,0.0423452892717168,9.675491373645282e-06,0.9130638704757222,0.06587949804222165,-0.008448510868190498,0.03710131849417052,0.039972517006009484
24.5012,0.03909586876660256,0.037622314256703496,9.190529393651988e-05,0.8682962266965917,0.14703316762852828,0.002713993192156497,0.04051548257353673,0.043357434102399374
24.5012,0.03721730951738749,0.04133284190508545,1.9695848383783344e-05,0.9998899666544032,0.0803685482373733,0.09982630090307903,0.04010253515718411,0.04251084773781603
24.5012,0.039189510868174765,0.043190614120459526,-1.1646287888122653e-05,1.0312918722789348,0.0757509186180059,0.1740996685121308,0.04194996749110543,0.043790254593970034


In [33]:
full_tab = vstack([tab1, tab2])

In [34]:
full_tab

energy_recoil_keV,aH,C,m,scale,A,B,true_yield_sig,cor1_yield_sig
float64,float64,float64,float64,float64,float64,float64,float64,float64
24.5012,0.037259034286155075,0.03820656999918315,7.459537803714777e-05,0.9778066235294974,0.06887616665387132,0.1616482321094805,0.040563204164007674,0.04261415589080108
24.5012,0.03978345283249353,0.0423452892717168,9.675491373645282e-06,0.9130638704757222,0.06587949804222165,-0.008448510868190498,0.03710131849417052,0.039972517006009484
24.5012,0.03909586876660256,0.037622314256703496,9.190529393651988e-05,0.8682962266965917,0.14703316762852828,0.002713993192156497,0.04051548257353673,0.043357434102399374
24.5012,0.03721730951738749,0.04133284190508545,1.9695848383783344e-05,0.9998899666544032,0.0803685482373733,0.09982630090307903,0.04010253515718411,0.04251084773781603
24.5012,0.039189510868174765,0.043190614120459526,-1.1646287888122653e-05,1.0312918722789348,0.0757509186180059,0.1740996685121308,0.04194996749110543,0.043790254593970034
24.5012,0.037259034286155075,0.03820656999918315,7.459537803714777e-05,0.9778066235294974,0.06887616665387132,0.1616482321094805,0.040563204164007674,0.04261415589080108
24.5012,0.03978345283249353,0.0423452892717168,9.675491373645282e-06,0.9130638704757222,0.06587949804222165,-0.008448510868190498,0.03710131849417052,0.039972517006009484
24.5012,0.03909586876660256,0.037622314256703496,9.190529393651988e-05,0.8682962266965917,0.14703316762852828,0.002713993192156497,0.04051548257353673,0.043357434102399374
24.5012,0.03721730951738749,0.04133284190508545,1.9695848383783344e-05,0.9998899666544032,0.0803685482373733,0.09982630090307903,0.04010253515718411,0.04251084773781603


In [35]:
full_tab.write('data/cat.hdf5', format='hdf5', path='table')

In [80]:
'yield_accuracy_Erecoil_%.2f_keV_%s.h5' % (2.45, 'all')

'yield_accuracy_Erecoil_2.45_keV_all.h5'